#### Libraries

In [ ]:
import os
import re
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.stats import gmean


#### Function to parse logs

In [ ]:
# base_dir - prefix directory for all logs
# traces - trace names for which logs to parse (X axis)
# configs - configuration names (Labels)
# line_parser - callable that uses regex on a log file to extract required values
def parse_logs(base_dir, traces, configs, line_parser):
    parsed_values = {trace: {} for trace in traces}
    for trace in traces:
        for config in configs:
            log_file = base_dir + f"{trace}-{config}.log"
            parsed_values[trace][config] = line_parser(log_file)
    return parsed_values

#### Main plotting function

In [ ]:
# data - 
# confignames - labels of configuration
# xticks - Traces names
# xlabel - String denoting X axis label
# ylabel - String denoting Y axis label
# title - String denoting title of graph
# barwidth - optional parameter to control bar spacing
# addbaseline - optional parameter to control whether to add a dashed horizontal line at y = 1
def plot(data, confignames, xticks, xlabel, ylabel, title, barwidth=0.2, addbaseline=True):
    # Plotting the speedups
    plt.rcParams.update({'font.size': 10})
    x = np.arange(len(xticks))  # X axis positions for the traces
    width = barwidth  # Width of the bars

    fig, ax = plt.subplots(figsize=(10, 5))

    # Plot bars for each configuration
    for i, config in enumerate(confignames):
        _values = [data[tick][config] for tick in xticks]
        bars = ax.bar(x + i*width, _values, width, label=config)

        # Annotate bars with speedup values
        for bar in bars:
            height = bar.get_height()
            ax.annotate(f'{height:.2f}',  # Format the annotation to 2 decimal places
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3),  # Offset text slightly above the bar
                        textcoords="offset points",
                        ha='center', va='bottom', rotation=90)

    if(addbaseline)
    # Add a translucent dashed line at y=1.0 for the baseline
    ax.axhline(y=1.0, color='gray', linestyle='--', linewidth=1.5, alpha=0.7)


    # Add labels and titles
    ax.set_xlabel(xlabel)
    ax.set_ylim(0, 5.0)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.set_xticks(x + 3*width / 2)
    ax.set_xticklabels(trace_names, rotation=0)
    ax.legend()

    # Show the plot
    plt.tight_layout()
    plt.show()


#### Utility function to list all filenames with given extension in a directory

In [ ]:
def list_trace_names(directory, extension):
    file_names = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(extension):
                file_name_custom = ".".join(file.split(".")[:-2])
                file_names.append(file_name_custom)
    return file_names

### Plotting Speedup

##### Callable to parse ipc values from a given logfile path

In [ ]:
def extract_ipc(log_file):
    with open(log_file, 'r') as f:
        for line in f:
            if 'CPU 0 cumulative IPC' in line:
                ipc_value = float(re.search(r"IPC:\s([\d\.]+)", line).group(1))
                return ipc_value
    return None

#### V1

In [ ]:
base_dir_v1 = "../logs/SPEC-CPU/"
directory_path = "../traces/SPEC-CPU2017/"
file_extension = ".xz"
configs_v3 = ['baseline', 'isb-l2', 'ipcp-l1', 'ipcp-isbv1', 'ipcp-isbv2', 'ipcp-isbv3-ATAP-acc', 'ipcp-isbv3-MTAP-acc']
trace_names_v3  = list_trace_names(directory_path, file_extension)